In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
#list with paths to the files
names_list = ['CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf', 'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf']
#list with flat arrays after substracting the dark current 
flats = []
# opne hdf file 
for i in range(len(names_list)): 
    print(i)
    fname = names_list[i]
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from pyhdf.SD import SD, SDC
import pandas as pd
from io import BytesIO

# Configure matplotlib backend
plt.switch_backend('TkAgg')
base_dir = 'CCD_gain'

def process_hdf_(file_path):
    

class ROIAnalyzer:
    def __init__(self, image, filename, year):
        self.fig, self.ax = plt.subplots(figsize=(10, 6))
        self.image = image
        self.filename = filename
        self.year = year
        self.rois = []
        self.stats = {}
        self.hist_image = None

        # Setup plot and selector
        self.ax.imshow(self.image, cmap='gray')
        self.ax.set_title(f"Select 2 ROIs\n{filename} ({year})")
        self.selector = RectangleSelector(
            self.ax,
            self.on_select,
            useblit=True,
            button=[1],
            minspanx=5,
            minspany=5,
            spancoords='pixels',
            interactive=True
        )
        plt.show(block=True)

    def on_select(self, eclick, erelease):
        if len(self.rois) >= 2:
            return
        
        x1 = int(min(eclick.xdata, erelease.xdata))
        x2 = int(max(eclick.xdata, erelease.xdata))
        y1 = int(min(eclick.ydata, erelease.ydata))
        y2 = int(max(eclick.ydata, erelease.ydata))
        self.rois.append((x1, y1, x2-x1, y2-y1))
        
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                           linewidth=1.5, edgecolor='red', facecolor='none')
        self.ax.add_patch(rect)
        self.fig.canvas.draw()
        
        if len(self.rois) == 2:
            self.process_data()
            plt.close()

    def process_data(self):
        combined_pixels = []
        for x, y, w, h in self.rois:
            try:
                roi_data = self.image[y:y+h, x:x+w]
                combined_pixels.extend(roi_data.flatten())
            except Exception as e:
                continue
        
        if combined_pixels:
            # Calculate statistics
            self.stats = {
                'mean': np.mean(combined_pixels),
                'median': np.median(combined_pixels),
                'std': np.std(combined_pixels),
                'total': len(combined_pixels)
            }
            
            # Generate and save histogram plot
            fig, ax = plt.subplots(figsize=(6, 4))
            ax.hist(combined_pixels, bins=50, color='blue', alpha=0.7, edgecolor='black')
            ax.set_title(f"Histogram\n{self.filename}")
            ax.set_xlabel("Pixel Value")
            ax.set_ylabel("Frequency")
            
            # Save plot to Bytes buffer
            buf = BytesIO()
            plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')
            plt.close(fig)
            self.hist_image = buf.getvalue()


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
#list with paths to the files
names_list = ['data/IAW_1000msSlowRamp_lighton_2.hdf', 'data/IAW_1000msSlowRamp_lighton_3.hdf','data/IAW_1000msSlowRamp_lightoff_2.hdf', 'data/IAW_1000msSlowRamp_lightoff_3.hdf']
#list with flat arrays after substracting the dark current 
flats = []
# opne hdf file 
for i in range(len(names_list)): 
    print(i)
    fname = names_list[i]
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

flats_add =[]
flats_diff = []

for i in range(0,len(flats),2):
    pair = flats[i:i+2]
 
    # Verify we have exactly 2 flats to average
    if len(pair) != 2:
        raise ValueError("Uneven number of flats for averaging")
        
    # Create averaged master flat
    avg_flat = (pair[0] + pair[1]) / 2.0
    flats_add.append(avg_flat)

    #difference
    diff = pair[0] -pair[1]
    flats_diff.append(diff)

diff_box =[]
roi_box = []
for i in range(len(flats_add)): 
    roi_size = 30
    y, x = 1024,1024
    full = flats_add[i]
    start_x = x//2 - roi_size//2
    start_y = y//2 - roi_size//2
    roi3= full[start_y:start_y+roi_size, start_x:start_x+roi_size]
    roi0 = full[200:300,100:200]
    roi2 = full[350:500,700:850]
    roi1 = full[590:640,500:600]
    roi4 = full[575:625, 300:350]
    roi5 = full[650:750,400:440]
    roi_box.append(roi0)
    roi_box.append(roi1)
    roi_box.append(roi2)
    roi_box.append(roi3)
    roi_box.append(roi4)
    roi_box.append(roi5)
    
    d = flats_diff[i]
    dmini3 = d[start_y:start_y+roi_size, start_x:start_x+roi_size]
    dmini0 = d[200:300,100:200]
    dmini2 = d[350:500,700:850]
    dmini1 = d[590:640,500:600]
    dmini4 = d[575:625,300:350]
    dmini5 = d[650:750,400:440]
    diff_box.append(dmini0)
    diff_box.append(dmini1)
    diff_box.append(dmini2)
    diff_box.append(dmini3)
    diff_box.append(dmini4)
    diff_box.append(dmini5)

variances_100 = []
means_100 = []
for i in range(len(diff_box)):

    stdev = np.std(diff_box[i])
    variance =int((stdev **2)/2)
    variances_100.append(variance)
    mean = int(np.mean(roi_box[i]))
    means_100.append(mean)
print(variances_100, "variances")
print(means_100, "means")

coordinates = list(zip(means_100,variances_100))
print(coordinates)

# Extract x and y values from the coordinates
x = [point[0] for point in coordinates]
y = [point[1] for point in coordinates]

# Plot the coordinates with markers and a connecting line
plt.plot(x, y, 'o', color='blue', markersize=8)
plt.xlabel('Mean')
plt.ylabel('Varaince')
plt.title('1000ms Lights on ')
plt.grid(True)
plt.show()

if len(coordinates) < 2:
    print("Error: At least two points are required to compute a slope.")
else:
    x = np.array([point[0] for point in coordinates])
    y = np.array([point[1] for point in coordinates])
    
    if np.all(x == x[0]):
        print("The line is vertical; slope is undefined.")
    else:
        # Calculate slope and intercept using linear regression
        slope, intercept = np.polyfit(x, y, 1)
        print(f"The slope of the line is {slope:.2f}")
        
        # Plotting the points and the line
        plt.scatter(x, y, color='red', label='Data Points')
        plt.plot(x, slope * x + intercept, label=f'Line: y = {slope:.2f}x + {intercept:.2f}')
        plt.legend()
        plt.xlabel('Mean')
        plt.ylabel('Varaince')
        plt.title('1000ms Lights on ')
        plt.grid(True)
        plt.show()


0


HDF4Error: SD: no such file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
#list with paths to the files
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
]
#list with flat arrays after substracting the dark current 
flats = []
# opne hdf file 
for i in range(len(names_list)): 
    print(i)
    fname = names_list[i]
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

flats_add =[]
flats_diff = []

for i in range(0,len(flats),2):
    pair = flats[i:i+2]
 
    # Verify we have exactly 2 flats to average
    if len(pair) != 2:
        raise ValueError("Uneven number of flats for averaging")
        
    # Create averaged master flat
    avg_flat = (pair[0] + pair[1]) / 2.0
    flats_add.append(avg_flat)

    #difference
    diff = pair[0] -pair[1]
    flats_diff.append(diff)

diff_box =[]
roi_box = []
for i in range(len(flats_add)): 
    roi_size = 30
    y, x = 1024,1024
    full = flats_add[i]
    start_x = x//2 - roi_size//2
    start_y = y//2 - roi_size//2
    roi3= full[start_y:start_y+roi_size, start_x:start_x+roi_size]
    roi0 = full[200:300,100:200]
    roi2 = full[350:500,700:850]
    roi1 = full[590:640,500:600]
    roi4 = full[575:625, 300:350]
    roi5 = full[650:750,400:440]
    roi_box.append(roi0)
    roi_box.append(roi1)
    roi_box.append(roi2)
    roi_box.append(roi3)
    roi_box.append(roi4)
    roi_box.append(roi5)
    
    d = flats_diff[i]
    dmini3 = d[start_y:start_y+roi_size, start_x:start_x+roi_size]
    dmini0 = d[200:300,100:200]
    dmini2 = d[350:500,700:850]
    dmini1 = d[590:640,500:600]
    dmini4 = d[575:625,300:350]
    dmini5 = d[650:750,400:440]
    diff_box.append(dmini0)
    diff_box.append(dmini1)
    diff_box.append(dmini2)
    diff_box.append(dmini3)
    diff_box.append(dmini4)
    diff_box.append(dmini5)

variances_100 = []
means_100 = []
for i in range(len(diff_box)):

    stdev = np.std(diff_box[i])
    variance =int((stdev **2)/2)
    variances_100.append(variance)
    mean = int(np.mean(roi_box[i]))
    means_100.append(mean)
print(variances_100, "variances")
print(means_100, "means")

coordinates = list(zip(means_100,variances_100))
print(coordinates)

# Extract x and y values from the coordinates
x = [point[0] for point in coordinates]
y = [point[1] for point in coordinates]

# Plot the coordinates with markers and a connecting line
plt.plot(x, y, 'o', color='blue', markersize=8)
plt.xlabel('Mean')
plt.ylabel('Varaince')
plt.title('1000ms Lights on ')
plt.grid(True)
plt.show()

if len(coordinates) < 2:
    print("Error: At least two points are required to compute a slope.")
else:
    x = np.array([point[0] for point in coordinates])
    y = np.array([point[1] for point in coordinates])
    
    if np.all(x == x[0]):
        print("The line is vertical; slope is undefined.")
    else:
        # Calculate slope and intercept using linear regression
        slope, intercept = np.polyfit(x, y, 1)
        print(f"The slope of the line is {slope:.2f}")
        
        # Plotting the points and the line
        plt.scatter(x, y, color='red', label='Data Points')
        plt.plot(x, slope * x + intercept, label=f'Line: y = {slope:.2f}x + {intercept:.2f}')
        plt.legend()
        plt.xlabel('Mean')
        plt.ylabel('Varaince')
        plt.title('1000ms Lights on ')
        plt.grid(True)
        plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
from matplotlib.widgets import RectangleSelector

# List with paths to the files
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
]

# List with flat arrays after subtracting the dark current 
flats = []

# Open hdf files and process data
for fname in names_list:
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

flats_add = []
flats_diff = []

# Process pairs of flats
for i in range(0, len(flats), 2):
    pair = flats[i:i+2]
    if len(pair) != 2:
        raise ValueError("Uneven number of flats for averaging")
    avg_flat = (pair[0] + pair[1]) / 2.0
    flats_add.append(avg_flat)
    diff = pair[0] - pair[1]
    flats_diff.append(diff)

diff_box = []
roi_box = []

# Function to handle ROI selection
def onselect(eclick, erelease, roi_coords):
    x1 = int(eclick.xdata)
    y1 = int(eclick.ydata)
    x2 = int(erelease.xdata)
    y2 = int(erelease.ydata)
    x_start, x_end = sorted([x1, x2])
    y_start, y_end = sorted([y1, y2])
    roi_coords.append((y_start, y_end, x_start, x_end))
    if len(roi_coords) >= 5:
        plt.close()

# Loop through each averaged flat to select ROIs
for idx in range(len(flats_add)):
    current_flat = flats_add[idx]
    current_diff = flats_diff[idx]
    roi_coords = []
    
    fig, ax = plt.subplots()
    ax.imshow(current_flat, cmap='prism')
    ax.set_title(f'Pair {idx+1}: Select 5 ROIs, then close window')
    
    # Connect the RectangleSelector
    rs = RectangleSelector(ax, lambda eclick, erelease: onselect(eclick, erelease, roi_coords),
                           useblit=True, button=[1],
                           minspanx=5, minspany=5,
                           spancoords='pixels', interactive=True)
    
    plt.show(block=True)
    
    # Ensure exactly 5 ROIs are selected
    if len(roi_coords) != 5:
        raise ValueError(f"Exactly 5 ROIs must be selected for pair {idx+1}, but {len(roi_coords)} were provided.")
    
    # Extract ROIs from current_flat and current_diff
    for coords in roi_coords:
        y_start, y_end, x_start, x_end = coords
        roi = current_flat[y_start:y_end, x_start:x_end]
        roi_box.append(roi)
        d_roi = current_diff[y_start:y_end, x_start:x_end]
        diff_box.append(d_roi)

# Calculate variances and means
variances_100 = []
means_100 = []
for i in range(len(diff_box)):
    stdev = np.std(diff_box[i])
    variance = (stdev ** 2) / 2
    variances_100.append(variance)
    mean = np.mean(roi_box[i])
    means_100.append(mean)

coordinates = list(zip(means_100, variances_100))
#print(coordinates)

# Plotting
x = [point[0] for point in coordinates]
y = [point[1] for point in coordinates]

plt.plot(x, y, 'o', color='blue', markersize=8)
plt.xlabel('Mean')
plt.ylabel('Variance')
plt.title('1000ms Lights on')
plt.grid(True)
plt.show()

# Linear regression and plot
if len(coordinates) < 2:
    print("Error: At least two points are required to compute a slope.")
else:
    x = np.array([point[0] for point in coordinates])
    y = np.array([point[1] for point in coordinates])
    
    if np.all(x == x[0]):
        print("The line is vertical; slope is undefined.")
    else:
        slope, intercept = np.polyfit(x, y, 1)
        print(f"The slope of the line is {slope:.2f}")
        
        plt.scatter(x, y, color='red', label='Data Points')
        plt.plot(x, slope * x + intercept, label=f'Line: y = {slope:.2f}x + {intercept:.2f}')
        plt.legend()
        plt.xlabel('Mean')
        plt.ylabel('Variance')
        plt.title('1000ms Lights on')
        plt.grid(True)
        plt.show()

[(542.4407329643062, 4455.598559248203), (1384.063508064516, 5622.049326264535), (659.3079202586207, 6973.686738499941), (2396.3137351778655, 5336.589494660907), (1668.2239827856024, 13111.590939003103), (315.5466367102396, 1616.2735488483622), (237.59768457841852, 2343.7115507085473), (-7.218482252141983, 726.8440646119263), (239.7346394984326, 2142.1190399345314), (176.77564046895353, 1271.206069072246)]
The slope of the line is 3.21


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
from matplotlib.widgets import RectangleSelector

# List with paths to the files
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
]

# List with flat arrays after subtracting the dark current 
flats = []

# Open hdf files and process data
for fname in names_list:
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

flats_add = []
flats_diff = []

# Process pairs of flats
for i in range(0, len(flats), 2):
    pair = flats[i:i+2]
    if len(pair) != 2:
        raise ValueError("Uneven number of flats for averaging")
    avg_flat = (pair[0] + pair[1]) / 2.0
    flats_add.append(avg_flat)
    diff = pair[0] - pair[1]
    flats_diff.append(diff)

diff_box = []
roi_box = []

# Function to handle ROI selection
def onselect(eclick, erelease, roi_coords):
    x1 = int(eclick.xdata)
    y1 = int(eclick.ydata)
    x2 = int(erelease.xdata)
    y2 = int(erelease.ydata)
    x_start, x_end = sorted([x1, x2])
    y_start, y_end = sorted([y1, y2])
    roi_coords.append((y_start, y_end, x_start, x_end))
    if len(roi_coords) >= 5:
        plt.close()

# Loop through each averaged flat to select ROIs
for idx in range(len(flats_add)):
    current_flat = flats_add[idx]
    current_diff = flats_diff[idx]
    roi_coords = []
    
    fig, ax = plt.subplots()
    ax.imshow(current_flat, cmap='prism')
    ax.set_title(f'Pair {idx+1}: Select 5 ROIs, then close window')
    
    # Connect the RectangleSelector
    rs = RectangleSelector(ax, lambda eclick, erelease: onselect(eclick, erelease, roi_coords),
                           useblit=True, button=[1],
                           minspanx=5, minspany=5,
                           spancoords='pixels', interactive=True)
    
    plt.show(block=True)
    
    # Ensure exactly 5 ROIs are selected
    if len(roi_coords) != 5:
        raise ValueError(f"Exactly 5 ROIs must be selected for pair {idx+1}, but {len(roi_coords)} were provided.")
    
    # Extract ROIs from current_flat and current_diff
    for coords in roi_coords:
        y_start, y_end, x_start, x_end = coords
        roi = current_flat[y_start:y_end, x_start:x_end]
        roi_box.append(roi)
        d_roi = current_diff[y_start:y_end, x_start:x_end]
        diff_box.append(d_roi)

# Calculate variances and means
variances_100 = []
means_100 = []
stats_add = []
stats_diff = []
for i in range(len(diff_box)):
    stdev = np.std(diff_box[i])
    variance = (stdev ** 2) / 2
    variances_100.append(variance)
    mean = np.mean(roi_box[i])
    means_100.append(mean)

    # Calculate statistics
    stats_add.append({
        'mean': np.mean(roi_box[i]),
        'median': np.median(roi_box[i]),
        'std': np.std(roi_box[i]),
        'var': np.var(roi_box[i]),
        "avg": np.average(roi_box[i])
    })
    
    stats_diff.append({
        'mean': np.mean(diff_box[i]),
        'median': np.median(diff_box[i]),
        'std': np.std(diff_box[i]),
        'var': np.var(diff_box[i]),
        "avg": np.average(diff_box[i])
    })

# Print summary statistics
print("\nSummary Statistics:")
print(f"{'ROI #':<6} | {'Type':<6} | {'Mean':<10} | {'Average':<10} | {'Std Dev':<10} | {'Variance':<10} | {'Median':<10}")
print("-"*75)
for i in range(len(roi_box)):
    print(f"{i+1:<6} | {'Add':<6} | {stats_add[i]['mean']:10.2f} | {stats_add[i]['avg']:10.2f} |{stats_add[i]['std']:10.2f} | {stats_add[i]['var']:10.2f} | {stats_add[i]['median']:10.2f}")
    print(f"{i+1:<6} | {'Diff':<6} | {stats_diff[i]['mean']:10.2f} | {stats_diff[i]['avg']:10.2f} |{stats_diff[i]['std']:10.2f} | {stats_diff[i]['var']:10.2f} | {stats_diff[i]['median']:10.2f}")
    print("-"*75)


coordinates = list(zip(means_100, variances_100))
#print(coordinates)

# Plotting scattered points 
x = [point[0] for point in coordinates]
y = [point[1] for point in coordinates]

plt.plot(x, y, 'o', color='blue', markersize=8)
plt.xlabel('Mean')
plt.ylabel('Variance')
plt.title('1000ms Lights on')
plt.grid(True)
plt.show()

# Linear regression and plot and find slope 
if len(coordinates) < 2:
    print("Error: At least two points are required to compute a slope.")
else:
    x = np.array([point[0] for point in coordinates])
    y = np.array([point[1] for point in coordinates])
    
    if np.all(x == x[0]):
        print("The line is vertical; slope is undefined.")
    else:
        slope, intercept = np.polyfit(x, y, 1)
        print(f"The slope of the line is {slope:.2f}")
        
        plt.scatter(x, y, color='red', label='Data Points')
        plt.plot(x, slope * x + intercept, label=f'Line: y = {slope:.2f}x + {intercept:.2f}')
        plt.legend()
        plt.xlabel('Mean')
        plt.ylabel('Variance')
        plt.title('1000ms Lights on')
        plt.grid(True)
        plt.show()


Summary Statistics:
ROI #  | Type   | Mean       | Average    | Std Dev    | Variance   | Median    
---------------------------------------------------------------------------
1      | Add    |      17.82 |      17.82 |     33.09 |    1094.85 |      18.00
1      | Diff   |       1.29 |       1.29 |     40.94 |    1676.35 |       1.00
---------------------------------------------------------------------------
2      | Add    |     214.90 |     214.90 |     74.27 |    5516.19 |     213.50
2      | Diff   |      -2.37 |      -2.37 |     67.06 |    4496.91 |      -2.00
---------------------------------------------------------------------------
3      | Add    |     555.71 |     555.71 |    107.21 |   11495.04 |     534.50
3      | Diff   |      -0.74 |      -0.74 |     79.57 |    6331.40 |      -1.00
---------------------------------------------------------------------------
4      | Add    |    1364.54 |    1364.54 |    117.98 |   13919.93 |    1364.50
4      | Diff   |      16.25 |    

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
from matplotlib.widgets import RectangleSelector

# List with paths to the files
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf'
]

# List with flat arrays after subtracting the dark current 
flats = []

# Open hdf files and process data
for fname in names_list:
    hdf = SD(fname, SDC.READ)
    dataset = hdf.select('Streak_array')
    data = dataset[:,:].astype(np.float64)
    hdf.end()
    flat = data[0] - data[1]
    flats.append(flat)

roi_box = []

# Function to handle ROI selection
def onselect(eclick, erelease, roi_coords):
    x1 = int(eclick.xdata)
    y1 = int(eclick.ydata)
    x2 = int(erelease.xdata)
    y2 = int(erelease.ydata)
    x_start, x_end = sorted([x1, x2])
    y_start, y_end = sorted([y1, y2])
    roi_coords.append((y_start, y_end, x_start, x_end))
    if len(roi_coords) >= 5:
        plt.close()

# Loop through each averaged flat to select ROIs
for idx in range(len(flats)):
    current_flat = flats[idx]
    roi_coords = []
    
    fig, ax = plt.subplots()
    ax.imshow(current_flat, cmap='prism')
    ax.set_title(f'Pair {idx+1}: Select 5 ROIs, then close window')
    
    # Connect the RectangleSelector
    rs = RectangleSelector(ax, lambda eclick, erelease: onselect(eclick, erelease, roi_coords),
                           useblit=True, button=[1],
                           minspanx=5, minspany=5,
                           spancoords='pixels', interactive=True)
    
    plt.show(block=True)
    
    # Ensure exactly 5 ROIs are selected
    if len(roi_coords) != 5:
        raise ValueError(f"Exactly 5 ROIs must be selected for pair {idx+1}, but {len(roi_coords)} were provided.")
    
    # Extract ROIs from current_flat and current_diff
    for coords in roi_coords:
        y_start, y_end, x_start, x_end = coords
        roi = current_flat[y_start:y_end, x_start:x_end]
        roi_box.append(roi)

# Calculate variances and means
variances_100 = []
means_100 = []
stats_add = []
for i in range(len(roi_box)):
    stdev = np.std(roi_box[i])
    variance = (stdev ** 2) / 2
    variances_100.append(variance)
    mean = np.mean(roi_box[i])
    means_100.append(mean)

    # Calculate statistics
    stats_add.append({
        'mean': np.mean(roi_box[i]),
        'median': np.median(roi_box[i]),
        'std': np.std(roi_box[i]),
        'var': np.var(roi_box[i]),
        "avg": np.average(roi_box[i])
    })
    

# Print summary statistics
print("\nSummary Statistics:")
print(f"{'ROI #':<6} | {'Type':<6} | {'Mean':<10} | {'Average':<10} | {'Std Dev':<10} | {'Variance':<10} | {'Median':<10}")
print("-"*75)
for i in range(len(roi_box)):
    print(f"{i+1:<6} | {'Add':<6} | {stats_add[i]['mean']:10.2f} | {stats_add[i]['avg']:10.2f} |{stats_add[i]['std']:10.2f} | {stats_add[i]['var']:10.2f} | {stats_add[i]['median']:10.2f}")
    print("-"*75)


coordinates = list(zip(means_100, variances_100))
#print(coordinates)

# Plotting scattered points 
x = [point[0] for point in coordinates]
y = [point[1] for point in coordinates]

plt.plot(x, y, 'o', color='blue', markersize=8)
plt.xlabel('Mean')
plt.ylabel('Variance')
plt.title('1000ms Lights on')
plt.grid(True)
plt.show()

# Linear regression and plot and find slope 
if len(coordinates) < 2:
    print("Error: At least two points are required to compute a slope.")
else:
    x = np.array([point[0] for point in coordinates])
    y = np.array([point[1] for point in coordinates])
    
    if np.all(x == x[0]):
        print("The line is vertical; slope is undefined.")
    else:
        slope, intercept = np.polyfit(x, y, 1)
        print(f"The slope of the line is {slope:.2f}")
        
        plt.scatter(x, y, color='red', label='Data Points')
        plt.plot(x, slope * x + intercept, label=f'Line: y = {slope:.2f}x + {intercept:.2f}')
        plt.legend()
        plt.xlabel('Mean')
        plt.ylabel('Variance')
        plt.title('1000ms Lights on')
        plt.grid(True)
        plt.show()


Summary Statistics:
ROI #  | Type   | Mean       | Average    | Std Dev    | Variance   | Median    
---------------------------------------------------------------------------
1      | Add    |     591.04 |     591.04 |    137.45 |   18893.00 |     572.00
---------------------------------------------------------------------------
2      | Add    |    1360.59 |    1360.59 |    146.92 |   21586.06 |    1357.00
---------------------------------------------------------------------------
3      | Add    |     616.55 |     616.55 |    130.82 |   17112.58 |     605.00
---------------------------------------------------------------------------
4      | Add    |    1661.86 |    1661.86 |    120.77 |   14585.33 |    1663.00
---------------------------------------------------------------------------
5      | Add    |    2515.60 |    2515.60 |    125.09 |   15647.53 |    2514.00
---------------------------------------------------------------------------
6      | Add    |     237.53 |     237.53 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC


plt.switch_backend('TkAgg')
# Configuration parameters
ROI_SIZE = 30  # Size of square ROI in pixels
NUM_ROIS = 5    # Number of ROIs to select per image

# List of file paths
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf'
]

# Load and process HDF files
flats = []
for fname in names_list:
    try:
        hdf = SD(fname, SDC.READ)
        dataset = hdf.select('Streak_array')
        data = dataset[:, :].astype(np.float64)
        hdf.end()
        flat = data[0] - data[1]
        flats.append(flat)
    except Exception as e:
        print(f"Error processing {fname}: {str(e)}")
        raise

# Create averaged and difference images
flats_add = []
flats_diff = []
for i in range(0, len(flats), 2):
    pair = flats[i:i+2]
    if len(pair) != 2:
        raise ValueError("Uneven number of flats for averaging")
    
    flats_add.append((pair[0] + pair[1]) / 2.0)
    flats_diff.append(pair[0] - pair[1])

# ROI selection and analysis
roi_add = []  # Stores ROIs from averaged images
roi_diff = []  # Stores corresponding ROIs from difference images
stats_add = []  # Statistics for averaged ROIs
stats_diff = []  # Statistics for difference ROIs

for idx in range(len(flats_add)):
    # Display image for ROI selection
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(flats_add[idx], cmap='viridis', origin='upper')
    ax.set_title(f"Select {NUM_ROIS} ROIs (click centers) | Image {idx+1}/{len(flats_add)}")
    plt.tight_layout()
    
    # Get ROI centers from user clicks
    print(f"Click {NUM_ROIS} locations for image {idx+1}")
    centers = plt.ginput(NUM_ROIS, timeout=0)
    plt.close()
    
    # Process each selected ROI
    for i, (x, y) in enumerate(centers):
        # Convert click coordinates to array indices
        col = int(round(x))
        row = int(round(y))
        
        # Calculate ROI boundaries with edge protection
        height, width = flats_add[idx].shape
        half = ROI_SIZE // 2
        
        row_start = max(0, row - half)
        row_end = min(height, row + half + (ROI_SIZE % 2))
        col_start = max(0, col - half)
        col_end = min(width, col + half + 70 + (ROI_SIZE % 2))
        
        # Extract ROIs from both image types
        roi_a = flats_add[idx][row_start:row_end, col_start:col_end]
        roi_d = flats_diff[idx][row_start:row_end, col_start:col_end]
        
        # Store ROIs
        roi_add.append(roi_a)
        roi_diff.append(roi_d)
        
        # Calculate statistics
        stats_add.append({
            'mean': np.mean(roi_a),
            'median': np.median(roi_a),
            'std': np.std(roi_a),
            'var': np.var(roi_a),
            "avg": np.average(roi_a)
        })
        
        stats_diff.append({
            'mean': np.mean(roi_d),
            'median': np.median(roi_d),
            'std': np.std(roi_d),
            'var': np.var(roi_d),
            "avg": np.average(roi_d)
        })

# Print summary statistics
print("\nSummary Statistics:")
print(f"{'ROI #':<6} | {'Type':<6} | {'Mean':<10} | {'Average':<10} | {'Std Dev':<10} | {'Variance':<10} | {'Median':<10}")
print("-"*75)
for i in range(len(roi_add)):
    print(f"{i+1:<6} | {'Add':<6} | {stats_add[i]['mean']:10.2f} | {stats_add[i]['avg']:10.2f} |{stats_add[i]['std']:10.2f} | {stats_add[i]['var']:10.2f} | {stats_add[i]['median']:10.2f}")
    print(f"{i+1:<6} | {'Diff':<6} | {stats_diff[i]['mean']:10.2f} | {stats_diff[i]['avg']:10.2f} |{stats_diff[i]['std']:10.2f} | {stats_diff[i]['var']:10.2f} | {stats_diff[i]['median']:10.2f}")
    print("-"*75)

# Optional: Save ROIs and statistics
np.savez('roi_data.npz', 
         roi_add=roi_add,
         roi_diff=roi_diff,
         stats_add=stats_add,
         stats_diff=stats_diff)

Click 5 locations for image 1

Summary Statistics:
ROI #  | Type   | Mean       | Average    | Std Dev    | Variance   | Median    
---------------------------------------------------------------------------
1      | Add    |       0.99 |       0.99 |     33.20 |    1102.24 |       1.25
1      | Diff   |      12.11 |      12.11 |     40.76 |    1661.32 |      13.00
---------------------------------------------------------------------------
2      | Add    |     874.56 |     874.56 |    110.58 |   12228.87 |     862.50
2      | Diff   |    1314.73 |    1314.73 |    189.00 |   35720.24 |    1296.50
---------------------------------------------------------------------------
3      | Add    |      17.96 |      17.96 |     33.68 |    1134.21 |      18.50
3      | Diff   |      34.93 |      34.93 |     39.34 |    1547.54 |      35.00
---------------------------------------------------------------------------
4      | Add    |     733.34 |     733.34 |     70.36 |    4950.52 |     731.25
4   

In [ ]:
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
]

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pyhdf.SD import SD, SDC
import os

plt.switch_backend('TkAgg')

class ROISelector:
    def __init__(self, image, num_rois=5):
        self.image = image
        self.num_rois = num_rois
        self.rois = []
        self.current_roi = None
        self.start_point = None
        self.fig, self.ax = plt.subplots(figsize=(12, 12))
        self.ax.imshow(self.image, cmap='gray', origin='upper', vmax=np.percentile(self.image, 99))
        self.ax.set_title(f'Select {self.num_rois} ROIs (click & drag)')
        
        # Connect event handlers
        self.cid_press = self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        self.cid_release = self.fig.canvas.mpl_connect('button_release_event', self.on_release)
        self.cid_motion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)

    def on_press(self, event):
        if event.inaxes != self.ax or len(self.rois) >= self.num_rois:
            return
        self.start_point = (event.xdata, event.ydata)
        self.current_roi = Rectangle((0,0), 0,0, 
                                   edgecolor='r', facecolor='none', 
                                   linewidth=1.5)
        self.ax.add_patch(self.current_roi)

    def on_motion(self, event):
        if self.current_roi is None or self.start_point is None:
            return
        start_x, start_y = self.start_point
        current_x = event.xdata or start_x
        current_y = event.ydata or start_y
        
        width = current_x - start_x
        height = current_y - start_y
        
        self.current_roi.set_width(width)
        self.current_roi.set_height(height)
        self.current_roi.set_xy((start_x, start_y))
        self.fig.canvas.draw()

    def on_release(self, event):
        if self.current_roi is None or self.start_point is None:
            return
        
        start_x, start_y = self.start_point
        end_x = event.xdata or start_x
        end_y = event.ydata or start_y
        
        # Convert to array coordinates
        x1 = int(np.clip(min(start_x, end_x), 0, self.image.shape[1]-1))
        x2 = int(np.clip(max(start_x, end_x), 0, self.image.shape[1]-1))
        y1 = int(np.clip(min(start_y, end_y), 0, self.image.shape[0]-1))
        y2 = int(np.clip(max(start_y, end_y), 0, self.image.shape[0]-1))
        
        if x2 > x1 and y2 > y1:
            self.rois.append((x1, x2, y1, y2))
            self.ax.text(x1, y1, str(len(self.rois)), 
                        color='r', fontsize=12, 
                        verticalalignment='top')
            
        self.current_roi = None
        self.start_point = None
        self.fig.canvas.draw()

    def get_rois(self):
        plt.close(self.fig)
        return self.rois

# ================== MAIN PROCESSING PIPELINE ================== #

# Configuration
names_list = [
    'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
    'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
    'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
]
NUM_ROIS = 5

# Verify files exist
for fname in names_list:
    if not os.path.exists(fname):
        raise FileNotFoundError(f"File not found: {fname}")

# Load and process HDF files
flats = []
print("\nProcessing HDF files:")
for fname in names_list:
    try:
        print(f"Loading {os.path.basename(fname)}...")
        hdf = SD(fname, SDC.READ)
        dataset = hdf.select('Streak_array')
        data = dataset[:,:].astype(np.float64)
        
        if data.shape[0] != 2:
            raise ValueError(f"Expected 2 frames, got {data.shape[0]}")
            
        # Create difference image (frame0 - frame1)
        flat = data[0] - data[1]
        flats.append(flat)
        hdf.end()
        
    except Exception as e:
        print(f"Error processing {fname}: {str(e)}")
        raise

# Create averaged and difference image pairs
flats_add = []
flats_diff = []
print("\nCreating image pairs:")
for i in range(0, len(flats), 2):
    pair = flats[i:i+2]
    if len(pair) != 2:
        raise ValueError(f"Missing pair for index {i} - uneven number of flats")
    
    avg_flat = (pair[0] + pair[1]) / 2.0
    diff_flat = pair[0] - pair[1]
    
    flats_add.append(avg_flat)
    flats_diff.append(diff_flat)
    print(f"Pair {i//2}: Created avg/diff images")

# ROI Selection and Analysis
roi_add = []
roi_diff = []
stats_add = []
stats_diff = []

print("\nStarting ROI selection:")
for idx, (add_img, diff_img) in enumerate(zip(flats_add, flats_diff)):
    print(f"\nProcessing image pair {idx+1}/{len(flats_add)}")
    
    # Interactive ROI selection
    selector = ROISelector(add_img, NUM_ROIS)
    plt.show()
    rois = selector.get_rois()
    
    # if len(rois) != NUM_ROIS:
    #     raise ValueError(f"Selected {len(rois)} ROIs instead of {NUM_ROIS}")
    
    # Process each ROI
    for roi_idx, (x1, x2, y1, y2) in enumerate(rois):
        # Extract from both image types
        add_roi = add_img[y1:y2, x1:x2]
        diff_roi = diff_img[y1:y2, x1:x2]
        
        # Store ROIs
        roi_add.append(add_roi)
        roi_diff.append(diff_roi)
        
        # Calculate statistics
        stats_add.append({
            'mean': np.mean(add_roi),
            'median': np.median(add_roi),
            'std': np.std(add_roi),
            'var': np.var(add_roi),
            'shape': add_roi.shape
        })
        
        stats_diff.append({
            'mean': np.mean(diff_roi),
            'median': np.median(diff_roi),
            'std': np.std(diff_roi),
            'var': np.var(diff_roi),
            'shape': diff_roi.shape
        })
        
        print(f"ROI {roi_idx+1}:")
        print(f"  Add stats: μ={stats_add[-1]['mean']:.2f}, σ={stats_add[-1]['std']:.2f}")
        print(f"  Diff stats: μ={stats_diff[-1]['mean']:.2f}, σ={stats_diff[-1]['std']:.2f}")

# Final Report
print("\n\nProcessing complete!")
print(f"Processed {len(flats_add)} image pairs")
print(f"Selected {len(roi_add)} total ROIs")
print("\nStatistics summary:")
print(f"{'Type':<6} | {'Mean':<10} | {'Std Dev':<10} | {'Variance':<10}")
print("-"*45)
print(f"{'Add':<6} | {np.mean([s['mean'] for s in stats_add]):<10.2f} | {np.mean([s['std'] for s in stats_add]):<10.2f} | {np.mean([s['var'] for s in stats_add]):<10.2f}")
print(f"{'Diff':<6} | {np.mean([s['mean'] for s in stats_diff]):<10.2f} | {np.mean([s['std'] for s in stats_diff]):<10.2f} | {np.mean([s['var'] for s in stats_diff]):<10.2f}")

# Save results
np.savez('ccd_analysis_results.npz',
         roi_add=roi_add,
         roi_diff=roi_diff,
         stats_add=stats_add,
         stats_diff=stats_diff,
         flats_add=flats_add,
         flats_diff=flats_diff)

print("\nResults saved to ccd_analysis_results.npz")


Processing HDF files:
Loading IAW_1000msSlowRamp_lighton_2.hdf...
Loading IAW_1000msSlowRamp_lighton_3.hdf...
Loading IAW_1000msSlowRamp_lightoff_3.hdf...
Loading IAW_1000msSlowRamp_lightoff_1.hdf...
Loading IAW_2000msSlowRamp_lighton_4.hdf...
Loading IAW_2000msSlowRamp_lighton_1.hdf...
Loading IAW_2000msSlowRamp_lightoff_2.hdf...
Loading IAW_2000msSlowRamp_lightoff_1.hdf...
Loading IAW_5000msSlowRamp_lighton_3.hdf...
Loading IAW_5000msSlowRamp_lighton_1.hdf...
Loading IAW_5000msSlowRamp_lightoff_2.hdf...
Loading IAW_5000msSlowRamp_lightoff_3.hdf...

Creating image pairs:
Pair 0: Created avg/diff images
Pair 1: Created avg/diff images
Pair 2: Created avg/diff images
Pair 3: Created avg/diff images
Pair 4: Created avg/diff images
Pair 5: Created avg/diff images

Starting ROI selection:

Processing image pair 1/6

Processing image pair 2/6

Processing image pair 3/6

Processing image pair 4/6

Processing image pair 5/6

Processing image pair 6/6


Processing complete!
Processed 6 image 

c:\Users\jher\AppData\Local\anaconda3\envs\tsadar-cpu\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jher\AppData\Local\anaconda3\envs\tsadar-cpu\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
import os
import gc
from matplotlib.widgets import RectangleSelector
#from matplotlib.pyplot import PyQt5, PySide2

# Configuration
NUM_ROIS = 5
DOWNSCALE_FACTOR = 4  # For display optimization
ROI_MIN_SIZE = 32

class ROIProcessor:
    def __init__(self):
        self.rois = []
        self.current_roi = None
        self.fig = None
        self.ax = None
        self.full_res_image = None
        
    def _downsample(self, image):
        """Downsample image for display while preserving original coordinates"""
        return image[::DOWNSCALE_FACTOR, ::DOWNSCALE_FACTOR]

    def _upscale_coords(self, x, y):
        """Convert display coordinates to original image coordinates"""
        return x * DOWNSCALE_FACTOR, y * DOWNSCALE_FACTOR

    def onselect(self, eclick, erelease):
        """Handle rectangle selection with coordinate conversion"""
        x1, y1 = self._upscale_coords(eclick.xdata, eclick.ydata)
        x2, y2 = self._upscale_coords(erelease.xdata, erelease.ydata)
        
        # Ensure valid ROI coordinates
        x1, x2 = sorted([x1, x2])
        y1, y2 = sorted([y1, y2])
        
        height, width = self.full_res_image.shape
        self.rois.append((
            max(0, int(x1)), min(width, int(x2)),
            max(0, int(y1)), min(height, int(y2))
        ))
        
        # Update display
        self.ax.add_patch(plt.Rectangle(
            (eclick.xdata, eclick.ydata),
            erelease.xdata - eclick.xdata,
            erelease.ydata - eclick.ydata,
            edgecolor='cyan', facecolor='none', linewidth=1.5
        ))
        self.fig.canvas.draw_idle()

    def select_rois(self, image):
        """Interactive ROI selection with optimized display"""
        self.full_res_image = image
        display_image = self._downsample(image)
        
        # Create figure with efficient backend
        plt.switch_backend('Qt5Agg')
        self.fig, self.ax = plt.subplots(figsize=(12, 12))
        self.ax.imshow(display_image, cmap='gray', 
                      vmax=np.percentile(display_image, 99.9))
        
        # Set up rectangle selector
        rs = RectangleSelector(self.ax, self.onselect,
                              useblit=True,
                              button=[1],
                              minspanx=ROI_MIN_SIZE//DOWNSCALE_FACTOR,
                              minspany=ROI_MIN_SIZE//DOWNSCALE_FACTOR,
                              spancoords='pixels',
                              interactive=False)
        
        plt.title(f'Select {NUM_ROIS} ROIs (click/drag) | Close when done')
        plt.tight_layout()
        
        # Keep open until we have enough ROIs
        while len(self.rois) < NUM_ROIS:
            plt.pause(0.1)
            
        plt.close()
        return self.rois[:NUM_ROIS]

def process_hdf_file(fname):
    """Efficient HDF file processing with memory cleanup"""
    try:
        hdf = SD(fname, SDC.READ)
        dataset = hdf.select('Streak_array')
        data = dataset[:,:].astype(np.float32)  # Use float32 for memory efficiency
        hdf.end()
        
        if data.shape[0] != 2:
            raise ValueError("Invalid frame count")
            
        # Process and immediately clear memory
        result = data[0] - data[1]
        del data
        gc.collect()
        return result 
        
    except Exception as e:
        print(f"Error processing {fname}: {str(e)}")
        raise

def main():
    # Configuration
    input_files = [
        'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
    ]
    
    # File validation
    missing = [f for f in input_files if not os.path.exists(f)]
    if missing:
        raise FileNotFoundError(f"Missing files: {missing}")

    # Process files in batches
    results = []
    for fname in input_files:
        print(f"Processing {os.path.basename(fname)}...")
        results.append(process_hdf_file(fname))
        gc.collect()

    # Pair processing with memory optimization
    processor = ROIProcessor()
    stats = {'add': [], 'diff': []}
    
    for i in range(0, len(results), 2):
        # Process pairs sequentially
        pair = results[i:i+2]
        if len(pair) != 2:
            raise ValueError("Incomplete pair")
            
        # Calculate pair data
        avg = (pair[0] + pair[1]) * 0.5
        diff = pair[0] - pair[1]
        
        # Interactive ROI selection
        rois = processor.select_rois(avg)
        
        # Process ROIs
        for x1, x2, y1, y2 in rois:
            # Calculate statistics
            add_roi = avg[y1:y2, x1:x2]
            diff_roi = diff[y1:y2, x1:x2]
            
            stats['add'].append({
                'mean': np.mean(add_roi),
                'std': np.std(add_roi),
                'var': np.var(add_roi)
            })
            
            stats['diff'].append({
                'mean': np.mean(diff_roi),
                'std': np.std(diff_roi),
                'var': np.var(diff_roi)
            })
            
        # Clean up memory
        del avg, diff
        gc.collect()

    print("\nProcessing complete. Statistics:")
    print(f"Average signal: {np.mean([s['mean'] for s in stats['add']]):.2f}")
    print(f"Average noise: {np.mean([s['mean'] for s in stats['diff']]):.2f}")

if __name__ == "__main__":
    main()

Processing IAW_1000msSlowRamp_lighton_2.hdf...
Processing IAW_1000msSlowRamp_lighton_3.hdf...
Processing IAW_1000msSlowRamp_lightoff_3.hdf...
Processing IAW_1000msSlowRamp_lightoff_1.hdf...
Processing IAW_2000msSlowRamp_lighton_4.hdf...
Processing IAW_2000msSlowRamp_lighton_1.hdf...
Processing IAW_2000msSlowRamp_lightoff_2.hdf...
Processing IAW_2000msSlowRamp_lightoff_1.hdf...
Processing IAW_5000msSlowRamp_lighton_3.hdf...
Processing IAW_5000msSlowRamp_lighton_1.hdf...
Processing IAW_5000msSlowRamp_lightoff_2.hdf...
Processing IAW_5000msSlowRamp_lightoff_3.hdf...

Processing complete. Statistics:
Average signal: 934.53
Average noise: 97.43


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
import os
import gc
from matplotlib.widgets import RectangleSelector

# Configuration
NUM_ROIS = 5
DOWNSCALE_FACTOR = 4
ROI_MIN_SIZE = 32

class ROIProcessor:
    def __init__(self):
        self.rois = []
        self.current_roi = None
        self.fig = None
        self.ax = None
        self.full_res_image = None
        
    def _downsample(self, image):
        return image[::DOWNSCALE_FACTOR, ::DOWNSCALE_FACTOR]

    def _upscale_coords(self, x, y):
        return x * DOWNSCALE_FACTOR, y * DOWNSCALE_FACTOR

    def onselect(self, eclick, erelease):
        x1, y1 = self._upscale_coords(eclick.xdata, eclick.ydata)
        x2, y2 = self._upscale_coords(erelease.xdata, erelease.ydata)
        
        x1, x2 = sorted([x1, x2])
        y1, y2 = sorted([y1, y2])
        
        height, width = self.full_res_image.shape
        self.rois.append((
            max(0, int(x1)), min(width, int(x2)),
            max(0, int(y1)), min(height, int(y2))
        ))
        
        self.ax.add_patch(plt.Rectangle(
            (eclick.xdata, eclick.ydata),
            erelease.xdata - eclick.xdata,
            erelease.ydata - eclick.ydata,
            edgecolor='cyan', facecolor='none', linewidth=1.5
        ))
        self.fig.canvas.draw_idle()

    def select_rois(self, image):
        self.full_res_image = image
        display_image = self._downsample(image)
        
        plt.switch_backend('Qt5Agg')
        self.fig, self.ax = plt.subplots(figsize=(12, 12))
        self.ax.imshow(display_image, cmap='gray', 
                      vmax=np.percentile(display_image, 99.9))
        
        rs = RectangleSelector(self.ax, self.onselect,
                              useblit=True,
                              button=[1],
                              minspanx=ROI_MIN_SIZE//DOWNSCALE_FACTOR,
                              minspany=ROI_MIN_SIZE//DOWNSCALE_FACTOR,
                              spancoords='pixels',
                              interactive=False)
        
        plt.title(f'Select {NUM_ROIS} ROIs (click/drag) | Close when done')
        plt.tight_layout()
        
        while len(self.rois) < NUM_ROIS:
            plt.pause(0.1)
            
        plt.close()
        return self.rois[:NUM_ROIS]

def process_hdf_file(fname):
    try:
        hdf = SD(fname, SDC.READ)
        dataset = hdf.select('Streak_array')
        data = dataset[:,:].astype(np.float32)
        hdf.end()
        
        if data.shape[0] != 2:
            raise ValueError("Invalid frame count")
            
        result = data[0] - data[1]
        del data
        gc.collect()
        return result
        
    except Exception as e:
        print(f"Error processing {fname}: {str(e)}")
        raise

def print_statistics_table(stats):
    print("\n{:<6} | {:<8} | {:<10} | {:<10} | {:<10} | {:<10}".format(
        "ROI#", "Type", "Mean", "Median", "Std Dev", "Variance"))
    print("-"*78)
    
    for idx, (add_stat, diff_stat) in enumerate(zip(stats['add'], stats['diff'])):
        print("{:<6} | {:<8} | {:<10.2f} | {:<10.2f} | {:<10.2f} | {:<10.2f}".format(
            idx+1, "Add", 
            add_stat['mean'], add_stat['median'],
            add_stat['std'], add_stat['var']))
        
        print("{:<6} | {:<8} | {:<10.2f} | {:<10.2f} | {:<10.2f} | {:<10.2f}".format(
            "", "Diff", 
            diff_stat['mean'], diff_stat['median'],
            diff_stat['std'], diff_stat['var']))
        print("-"*78)

def main():
    input_files = [
        'CCD_gain/IAW_1000msSlowRamp_lighton_2.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lighton_3.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lightoff_3.hdf',
        'CCD_gain/IAW_1000msSlowRamp_lightoff_1.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lighton_4.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lighton_1.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lightoff_2.hdf',
        'CCD_gain/IAW_2000msSlowRamp_lightoff_1.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lighton_3.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lighton_1.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lightoff_2.hdf',
        'CCD_gain/IAW_5000msSlowRamp_lightoff_3.hdf'
    ]
    
    # Validate files
    missing = [f for f in input_files if not os.path.exists(f)]
    if missing:
        raise FileNotFoundError(f"Missing files: {missing}")

    # Process files
    results = []
    for fname in input_files:
        print(f"Processing {os.path.basename(fname)}...")
        results.append(process_hdf_file(fname))
        gc.collect()

    # Process pairs and collect statistics
    processor = ROIProcessor()
    stats = {'add': [], 'diff': []}
    
    for pair_num in range(len(results)//2):
        idx = pair_num * 2
        pair = results[idx:idx+2]
        
        print(f"\nProcessing pair {pair_num+1}/{len(results)//2}")
        avg = (pair[0] + pair[1]) * 0.5
        diff = pair[0] - pair[1]
        
        # Show interactive plot for each pair
        rois = processor.select_rois(avg)
        
        # Process ROIs for this pair
        for x1, x2, y1, y2 in rois:
            add_roi = avg[y1:y2, x1:x2]
            diff_roi = diff[y1:y2, x1:x2]
            
            stats['add'].append({
                'mean': np.mean(add_roi),
                'median': np.median(add_roi),
                'std': np.std(add_roi),
                'var': np.var(add_roi)
            })
            
            stats['diff'].append({
                'mean': np.mean(diff_roi),
                'median': np.median(diff_roi),
                'std': np.std(diff_roi),
                'var': np.var(diff_roi)
            })
        
        # Clean up
        del avg, diff
        gc.collect()
        plt.close('all')  # Ensure previous plots are closed

    # Print detailed statistics table
    print_statistics_table(stats)

if __name__ == "__main__":
    main()

Processing IAW_1000msSlowRamp_lighton_2.hdf...
Processing IAW_1000msSlowRamp_lighton_3.hdf...
Processing IAW_1000msSlowRamp_lightoff_3.hdf...
Processing IAW_1000msSlowRamp_lightoff_1.hdf...
Processing IAW_2000msSlowRamp_lighton_4.hdf...
Processing IAW_2000msSlowRamp_lighton_1.hdf...
Processing IAW_2000msSlowRamp_lightoff_2.hdf...
Processing IAW_2000msSlowRamp_lightoff_1.hdf...
Processing IAW_5000msSlowRamp_lighton_3.hdf...
Processing IAW_5000msSlowRamp_lighton_1.hdf...
Processing IAW_5000msSlowRamp_lightoff_2.hdf...
Processing IAW_5000msSlowRamp_lightoff_3.hdf...

Processing pair 1/6

Processing pair 2/6

Processing pair 3/6

Processing pair 4/6

Processing pair 5/6

Processing pair 6/6

ROI#   | Type     | Mean       | Median     | Std Dev    | Variance  
------------------------------------------------------------------------------
1      | Add      | 5.55       | 5.50       | 38.65      | 1493.98   
       | Diff     | 0.59       | 1.00       | 44.11      | 1945.66   
--------------